In [1]:
'''
1. 訓練データと正解ラベルの用意
'''
import numpy as np
import torch

# XORゲートの入力値　0と1の組み合わせの行列(4, 2)
train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
# XORゲートの出力　正解ラベルの行列(4, 1)
label = np.array([[0], [1], [1], [0]])

# モデルに入力できるようにTensorオブジェクトに変換
train_x = torch.Tensor(train)
train_y = torch.Tensor(label)

In [2]:
'''
2. モデルの定義
'''
import torch.nn as nn

class MLP(nn.Module):
    '''多層パーセプトロン
    
    Attributes:
      l1(Linear) : 隠れ層
      l2(Linear) : 出力層
      a1(Sigmoid): 隠れ層の活性化関数
      a2(Sigmoid): 出力層の活性化関数
    '''
    def __init__(self, input_dim, hidden_dim, output_dim):
        '''モデルの初期化を行う
        
        Parameters:
          input_dim(int) : 入力する1データあたりの値の形状
          hidden_dim(int): 隠れ層のユニット数
          output_dim(int): 出力層のユニット数
          
        '''
        # スーパークラスの__init__()を実行
        super().__init__()
        # 隠れ層、活性化関数はシグモイド
        self.fc1 = nn.Linear(input_dim,  # 入力するデータのサイズ
                            hidden_dim)  # 隠れ層のニューロン数                            
        
        # 出力層、活性化関数はシグモイド
        self.fc2 = nn.Linear(hidden_dim, # 入力するデータのサイズ
                                         # (＝前層のニューロン数)
                            output_dim)  # 出力層のニューロン数
        
    def forward(self, x):
        '''MLPの順伝播処理を行う
        
        Parameters:
          x(ndarray(float32)):訓練データ、または検証データ
          
        Returns(float32):
          出力層からの出力値    
        '''
        # レイヤー、活性化関数に前ユニットからの出力を入力する
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [3]:
'''
3. モデルの生成
'''
# 使用可能なデバイス(CPUまたはGPU）を取得する
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルオブジェクトを生成し、使用可能なデバイスを設定する
model = MLP(2, 2, 1).to(device)

model # モデルの構造を出力

MLP(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)

In [4]:
'''
4. 損失関数とオプティマイザーの生成
'''
import torch.optim

# バイナリ用のクロスエントロピー誤差のオブジェクトを生成
criterion = nn.BCELoss()
# 勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [5]:
'''
5. 勾配降下アルゴリズムによるパラメーターの更新処理
'''
def train_step(x, t):
    '''バックプロパゲーションによるパラメーター更新を行う
    
    Parameters: x(ndarray(float32)):訓練データ
                t(ndarray(float32)):正解ラベル
                
    Returns:
      MLPの出力と正解ラベルのクロスエントロピー誤差
    '''
    model.train() # モデルを訓練(学習)モードにする
    outputs = model(x) # モデルの出力を取得
    loss = criterion(outputs, t) # 出力と正解ラベルの誤差から損失を取得
    optimizer.zero_grad() # 勾配を0で初期化（累積してしまうため）
    loss.backward()  # 逆伝播の処理(自動微分による勾配計算)
    optimizer.step() # 勾配降下法の更新式を適用してバイアス、重みを更新

    return loss

In [6]:
'''
6.モデルを使用して学習する
'''
# エポック数
epochs = 4000

# 学習を行う
for epoch in range(epochs):
    # 1エポックごとの損失を保持する変数
    epoch_loss = 0.
    
    # torch.Tensorオブジェクトにデバイスを割り当てる
    train_x, train_y = train_x.to(device), train_y.to(device)

    # データをモデルに入力し、バイアス、重みを更新して誤差を取得
    loss = train_step(train_x, train_y)
    epoch_loss += loss.item()

    # 1000エポックごとに結果を出力
    if (epoch + 1) % 1000 == 0:
        print('epoch({}) loss: {:.4f}'.format(epoch+1, epoch_loss))

epoch(1000) loss: 0.6931
epoch(2000) loss: 0.6929
epoch(3000) loss: 0.2126
epoch(4000) loss: 0.0209


In [7]:
'''
7. 学習済みのモデルで予測する
'''
# 学習済みのMLPの出力
outputs = model(train_x)
print(outputs)
# デバイスがCPUに設定されたTensorを取得し、これをNumPy配列に変換
# 出力値を閾値0.5で0と1に分類する
preds = (outputs.to('cpu').detach().numpy().copy() > 0.5
        ).astype(np.int32)
print(preds)

tensor([[0.0162],
        [0.9814],
        [0.9814],
        [0.0291]], grad_fn=<SigmoidBackward>)
[[0]
 [1]
 [1]
 [0]]
